**We validate the deployed model using:**

ISIC 2018 validation images - Ensures accuracy on standard clinical data
External images (Google/web sources) - Tests real-world robustness across different cameras, lighting, and skin types

Expected Results

Processing time: ~2-5 seconds per image
Output: Binary mask highlighting lesion boundaries
Visualization: Red overlay on original image for intuitive interpretation


Notebook Structure:

Setup & Installation
Model Architecture Definition
Model Loading & Initialization
Inference Function
Gradio Interface Deployment

Let's get started! 👇

In [1]:
# Install required packages (run this cell first)
!pip install torch torchvision albumentations pillow gradio -q


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, resnet101

import albumentations as A
from albumentations.pytorch import ToTensorV2

import gradio as gr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# ============ MODEL ARCHITECTURE ============
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        atrous_rates = [6, 12, 18]

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        self.conv2 = self._make_atrous_conv(in_channels, out_channels, atrous_rates[0])
        self.conv3 = self._make_atrous_conv(in_channels, out_channels, atrous_rates[1])
        self.conv4 = self._make_atrous_conv(in_channels, out_channels, atrous_rates[2])

        self.global_avg_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        self.conv_fusion = nn.Sequential(
            nn.Conv2d(out_channels * 5, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )

    def _make_atrous_conv(self, in_channels, out_channels, dilation):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=dilation,
                      dilation=dilation, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        size = x.shape[2:]
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x4 = self.conv4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=size, mode='bilinear', align_corners=True)
        x = torch.cat([x1, x2, x3, x4, x5], dim=1)
        return self.conv_fusion(x)


class Decoder(nn.Module):
    def __init__(self, low_level_channels, num_classes):
        super(Decoder, self).__init__()

        self.conv_low_level = nn.Sequential(
            nn.Conv2d(low_level_channels, 48, 1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True)
        )

        self.conv_decoder = nn.Sequential(
            nn.Conv2d(304, 304, 3, padding=1, groups=304, bias=False),
            nn.BatchNorm2d(304),
            nn.ReLU(inplace=True),
            nn.Conv2d(304, 256, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),

            nn.Conv2d(256, 256, 3, padding=1, groups=256, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1)
        )

        self.classifier = nn.Conv2d(256, num_classes, 1)

    def forward(self, x, low_level_feat):
        low_level_feat = self.conv_low_level(low_level_feat)
        x = F.interpolate(x, size=low_level_feat.shape[2:],
                          mode='bilinear', align_corners=True)
        x = torch.cat([x, low_level_feat], dim=1)
        x = self.conv_decoder(x)
        return self.classifier(x)


class DeepLabV3Plus(nn.Module):
    def __init__(self, num_classes=1, backbone='resnet50',
                 pretrained=True, output_stride=16):
        super(DeepLabV3Plus, self).__init__()

        self.num_classes = num_classes
        self.output_stride = output_stride

        if backbone == 'resnet50':
            resnet = resnet50(pretrained=pretrained)
        elif backbone == 'resnet101':
            resnet = resnet101(pretrained=pretrained)
        else:
            raise ValueError(f"Unsupported backbone: {backbone}")

        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool

        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4

        if output_stride == 16:
            self._modify_resnet_layer(self.layer4, dilation=2)
        elif output_stride == 8:
            self._modify_resnet_layer(self.layer3, dilation=2)
            self._modify_resnet_layer(self.layer4, dilation=4)

        self.aspp = ASPP(in_channels=2048, out_channels=256)
        self.decoder = Decoder(low_level_channels=256, num_classes=num_classes)

        self._init_weights()

    def _modify_resnet_layer(self, layer, dilation):
        for block in layer:
            block.conv2.dilation = (dilation, dilation)
            block.conv2.padding = (dilation, dilation)

    def _init_weights(self):
        for m in self.decoder.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        input_size = x.shape[2:]

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        low_level_feat = self.layer1(x)

        x = self.layer2(low_level_feat)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.aspp(x)
        x = self.decoder(x, low_level_feat)

        x = F.interpolate(x, size=input_size, mode='bilinear', align_corners=True)
        return x


# ============ LOAD MODEL ============
print("\n" + "="*60)
print("Loading Model...")
print("="*60)

# Define model path - UPDATE THIS PATH!
MODEL_PATH = "/content/drive/MyDrive/Project/best_model.pth"

# Check if file exists
if not os.path.exists(MODEL_PATH):
    print(f"\n❌ ERROR: Model file not found at: {MODEL_PATH}")
    print("\nChecking your Drive structure...")

    # Try to find the file
    drive_root = "/content/drive/MyDrive"
    if os.path.exists(drive_root):
        print(f"\nContents of MyDrive:")
        for item in os.listdir(drive_root)[:20]:
            print(f"  - {item}")

    raise FileNotFoundError(f"Please update MODEL_PATH to the correct location of best_model.pth")

print(f"✓ Found model at: {MODEL_PATH}")

# Initialize model
model = DeepLabV3Plus(
    num_classes=1,
    backbone='resnet50',
    pretrained=False,
    output_stride=16
).to(device)

# Load weights
checkpoint = torch.load(MODEL_PATH, map_location=device)
if "model_state_dict" in checkpoint:
    model.load_state_dict(checkpoint["model_state_dict"])
else:
    model.load_state_dict(checkpoint)

model.eval()
print("✓ Model loaded successfully!\n")


# ============ PREPROCESSING ============
transform = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])



Loading Model...
✓ Found model at: /content/drive/MyDrive/Project/best_model.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✓ Model loaded successfully!



**Model Deployment**
Overview
Deploying our trained DeepLabV3+ model as an interactive web application using Gradio. Users can upload dermoscopic images and get instant lesion segmentation results.
Testing Strategy
We test the deployed model with images from two sources:
1. ISIC 2018 Dataset - Validation images (unseen during training) to verify accuracy on standard dermoscopic imaging
2. External Images - Real-world images from Google/medical websites to test robustness on varied cameras, lighting, and skin types
This ensures our model works both on clinical-grade data and real-world scenarios.
What You'll Get

Original Image - Your uploaded image
Binary Mask - White = lesion, Black = background
Overlay - Lesion highlighted in red

Run the cell below to launch the interface!

In [ ]:
# ============ INFERENCE FUNCTION ============
def predict_lesion(image):
    """
    Segments skin lesion from input image

    Args:
        image: PIL Image

    Returns:
        original, mask, overlay images
    """
    try:
        # Ensure RGB
        if image.mode != 'RGB':
            image = image.convert('RGB')

        original = image.copy()
        orig_size = original.size

        # Preprocess
        img_array = np.array(original)
        transformed = transform(image=img_array)
        img_tensor = transformed["image"].unsqueeze(0).to(device)

        # Inference
        with torch.no_grad():
            output = model(img_tensor)
            prob = torch.sigmoid(output)[0, 0].cpu().numpy()

        # Create mask
        binary_mask = (prob > 0.5).astype(np.uint8) * 255
        mask_pil = Image.fromarray(binary_mask, mode='L')
        mask_pil = mask_pil.resize(orig_size, Image.NEAREST)

        # Create overlay
        orig_array = np.array(original)
        mask_array = np.array(mask_pil)

        overlay_array = orig_array.copy()
        lesion_pixels = mask_array > 127

        # Blend red color with original
        overlay_array[lesion_pixels] = (
            orig_array[lesion_pixels] * 0.4 +
            np.array([255, 0, 0]) * 0.6
        ).astype(np.uint8)

        overlay_pil = Image.fromarray(overlay_array)

        return original, mask_pil, overlay_pil

    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        raise


# ============ GRADIO INTERFACE ============
print("="*60)
print("Creating Gradio Interface...")
print("="*60 + "\n")

interface = gr.Interface(
    fn=predict_lesion,
    inputs=gr.Image(type="pil", label="Upload Dermoscopic Image"),
    outputs=[
        gr.Image(type="pil", label="Original Image"),
        gr.Image(type="pil", label="Segmentation Mask (Binary)"),
        gr.Image(type="pil", label="Overlay (Red = Lesion)")
    ],
    title="🔬 Skin Lesion Segmentation - DeepLabV3+",
    description="""
    ### ISIC 2018 Challenge - Automated Skin Lesion Boundary Detection

    **How to use:**
    1. Upload a dermoscopic image of a skin lesion
    2. The model will automatically segment the lesion boundary
    3. View results: Original, Binary Mask, and Highlighted Overlay

    **Model:** DeepLabV3+ with ResNet-50 backbone
    """,
    allow_flagging="never",
    theme="soft"
)

# Launch interface
print("🚀 Launching Gradio UI...\n")
interface.launch(
    share=True,      # Creates public URL (lasts 72 hours)
    debug=True       # Shows detailed errors
)

Creating Gradio Interface...



/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


🚀 Launching Gradio UI...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a45728380bf6444a59.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-2975507468.py:32: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask_pil = Image.fromarray(binary_mask, mode='L')
/tmp/ipython-input-2975507468.py:32: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask_pil = Image.fromarray(binary_mask, mode='L')
/tmp/ipython-input-2975507468.py:32: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask_pil = Image.fromarray(binary_mask, mode='L')
/tmp/ipython-input-2975507468.py:32: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask_pil = Image.fromarray(binary_mask, mode='L')
/tmp/ipython-input-2975507468.py:32: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  mask_pil = Image.fromarray(binary_mask, mode='L')
/tmp/ipython-input-2975507468.py:32: DeprecationWarning: 'mode' parame